# Hello World for LLM

## Demo code to connect to Azure OpenAI
A quick demo code to connect to Azure OpenAI interface, and run some prompt.

Please ensure you have .env file in your HOME/Documents/src/openai/ folder

In [8]:
from openai import AzureOpenAI
from dotenv import load_dotenv
import os

env_path = os.getenv("HOME") + "/Documents/src/openai/.env"
print("env_path is ",env_path)
load_dotenv(dotenv_path=env_path, verbose=True)

os.environ["OPENAI_API_TYPE"] = "azure"
os.environ["OPENAI_API_VERSION"] = "2023-05-15"
os.environ["AZURE_OPENAI_ENDPOINT"] = "https://pvg-azure-openai-uk-south.openai.azure.com"

client = AzureOpenAI(
  azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT"), 
  api_key=os.getenv("AZURE_OPENAI_API_KEY"),  
  api_version="2023-05-15"
)

response = client.chat.completions.create(
    model="gpt-35-turbo", # model = "deployment_name".
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "Do other Azure AI services support this too?"}
    ]
)

print(response.choices[0].message.content)

env_path is  /Users/I069899/Documents/src/openai/.env
Yes, many Azure AI services support a similar level of functionality and integration with other Azure services. Some common examples include Azure Cognitive Services, Azure Machine Learning, and Azure Bot Service, among others. Each of these services offers a unique set of capabilities designed to meet various AI and machine learning needs. If you have specific requirements or questions about other Azure AI services, feel free to ask!


In [2]:
from langchain.chat_models import AzureChatOpenAI
from langchain.schema import HumanMessage
 
model = AzureChatOpenAI(deployment_name="gpt-35-turbo")
 
message = HumanMessage(
    content="Translate this sentence from English to Chinese: Who am I?"
)
print(model([message]))

/Users/I069899/anaconda3/envs/py310/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.chat_models.azure_openai.AzureChatOpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import AzureChatOpenAI`.
  warn_deprecated(
/Users/I069899/anaconda3/envs/py310/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.7 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


content='我是谁？ (Wǒ shì shuí?)'


## Count the token of an input

A piece of code to visualize the token of a string input. 

In [1]:
import tiktoken
from IPython.display import clear_output

enc4GPT4 = tiktoken.encoding_for_model("gpt-4")
print(enc4GPT4)
color_codes = ['0;47', '0;42', '0;43', '0;44', '0;46', '0;45']

user_input = input("Input your text: ")
clear_output(wait=True)

encoded = enc4GPT4.encode(user_input)
decoded = enc4GPT4.decode_tokens_bytes(encoded)
token_list = [token.decode() for token in decoded]

char_count= sum(len(i) for i in token_list)

for idx, token in enumerate(token_list):
    print(f'\x1b[{color_codes[idx % len(color_codes)]};1m{token}\x1b[0m', end='')

print("\n\n" + str(token_list) + "\n")
print(str(encoded) + "\n")
print("Token Count: " + str(len(encoded)))
print("Characters: "  + str(char_count))

In today's rapidly changing world, knowledge is constantly being refreshed, and it's essential to adapt and evolve in order to avoid becoming obsolete. The impact of macro-economic factors on our company and industry necessitates a proactive approach skill development and strategic influence to ensure that our team remains at the forefront of our organization. By leveraging our collective skills and expertise, we have the opportunity to consolidate efforts across multiple teams and achieve larger, more impactful goals. While individuals may excel in their own right, it's through collaborative efforts that we can truly propel our path forward.

['In', ' today', "'s", ' rapidly', ' changing', ' world', ',', ' knowledge', ' is', ' constantly', ' being', ' refreshed', ',', ' and', ' it', "'s", ' essential', ' to', ' adapt', ' and', ' evolve', ' in', ' order', ' to', ' avoid', ' becoming', ' obsolete', '.', ' The', ' impact', ' of', ' macro', '-economic', ' factors', ' on', ' our', ' compan

Here is a piece of sample code to avoid token exceeding limits.

In [9]:
from openai import AzureOpenAI
from dotenv import load_dotenv
import os

env_path = os.getenv("HOME") + "/Documents/src/openai/.env"
print("env_path is ",env_path)
load_dotenv(dotenv_path=env_path, verbose=True)

client = AzureOpenAI(
  azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT"), 
  api_key=os.getenv("AZURE_OPENAI_API_KEY"),  
  api_version="2023-05-15"
)

system_message = {"role": "system", "content": "You are a helpful assistant."}
max_response_tokens = 250
token_limit= 500
conversation=[]
conversation.append(system_message)


def num_tokens_from_messages(messages, model="gpt-3.5-turbo-0301"):
    encoding = tiktoken.encoding_for_model(model)
    num_tokens = 0
    for message in messages:
        num_tokens += 4  # every message follows <im_start>{role/name}\n{content}<im_end>\n
        for key, value in message.items():
            num_tokens += len(encoding.encode(value))
            if key == "name":  # if there's a name, the role is omitted
                num_tokens += -1  # role is always required and always 1 token
    num_tokens += 2  # every reply is primed with <im_start>assistant
    return num_tokens

while(True):
    user_input = input("")     
    conversation.append({"role": "user", "content": user_input})
    conv_history_tokens = num_tokens_from_messages(conversation)

    while (conv_history_tokens+max_response_tokens >= token_limit):
        del conversation[1]
        conv_history_tokens = num_tokens_from_messages(conversation)
        
    response = client.chat.completions.create(
        model="gpt-35-turbo",
        messages = conversation,
        temperature=.7,
        max_tokens=max_response_tokens,
    )

    conversation.append({"role": "assistant", "content": response.choices[0].message.content})
    print("\n" + response.choices[0].message.content + "\n")
    post_response=num_tokens_from_messages(conversation)
    print(f"Post response token count: {post_response}")
    print("\n" + str(conversation[1]))
    print("\n\n")

env_path is  /Users/I069899/Documents/src/openai/.env

I'm an AI digital assistant, so I don't have real-time information. However, you can easily check the weather by using a weather website or app.

Post response token count: 63

{'role': 'user', 'content': 'Hello, how is the weather today?'}




To check the weather using a weather app, you can follow these steps:

1. Open the app store on your smartphone (e.g., Google Play Store for Android or the App Store for iOS).
2. In the search bar, type in "weather app" and search for available options.
3. Choose a weather app that has good ratings and reviews.
4. Download and install the app on your smartphone.
5. Once the app is installed, open it and allow location access so that it can provide you with the weather information for your current location.
6. You can then use the app to view the current weather conditions, hourly forecasts, and extended forecasts for your area.

Popular weather apps include AccuWeather, The Weather Channel, 

In [1]:
from openai import AzureOpenAI
from dotenv import load_dotenv
import pandas as pd
import tiktoken
import os
from IPython.display import clear_output

env_path = os.getenv("HOME") + "/Documents/src/openai/.env"
load_dotenv(dotenv_path=env_path, verbose=True)

os.environ["OPENAI_API_TYPE"] = "azure"
os.environ["OPENAI_API_VERSION"] = "2023-05-15"
os.environ["AZURE_OPENAI_ENDPOINT"] = "https://pvg-azure-openai-uk-south.openai.azure.com"



enc4Ada002 = tiktoken.encoding_for_model("text-embedding-ada-002")
print("**** enc4Ada002 is **** ", enc4Ada002)
enc4Emb3 = tiktoken.encoding_for_model("text-embedding-3-small")
print("**** enc4Emb3 is ***** ", enc4Emb3)

**** enc4Ada002 is ****  <Encoding 'cl100k_base'>
**** enc4Emb3 is *****  <Encoding 'cl100k_base'>
